## Record-pair Linkage Predictive Modeling Program

### Author: CDC's Data Science Upskilling (DSU) Team named, "MYABC 4 PPRL"
### Membership: DSU 2021-2022 (Cohort 3)
#### Team Project: Privacy Preserving Record Linkage (PPRL)


Python Program Description: 
This NoteBook contains ## Two model performance improvement activities are in this file: 
1.) Feature selection
2.) Parameter tuning


### Import the necessary packages

In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt  # works with seaborn etc to display plots created by seaborn etc
from matplotlib import style # to make the output of matplotlib pretier (all charts then turned red)
style.use("ggplot")# used because I prefer blue color for my charts
plt.rcParams["font.family"] = "Times New Roman"

import seaborn as sns

import plotly.express as plotExp
import plotly.figure_factory as factory_fig
import plotly.graph_objs as graphObj

from sklearn.preprocessing import MinMaxScaler # would handle outliers (data standardization)
from sklearn.preprocessing import LabelEncoder # for transforming Categorical data to Numerical data

#import time

In [ ]:
## So we could display our plots in a cell

%matplotlib inline
from pylab import rcParams
rcParams ['figure.figsize']=10,8

Let's utilize the interactive shell of Jupyter to ensure we could print multiple outplut from a cell, thus all output would come through from Ipython

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

#### Modify some of the Pandas printing capabilities

In [ ]:
# By default, Pandas will display only 10 rows/cols

# Globally set Max rows displayed in output to 20
pd.set_option('display.max_rows', 30)

# Globally set display for Pandas to always display all the columns
#pd.set_option('display.line_width', 5000)
pd.set_option('display.max_columns', None)# Settig to none makes display unlimited, better than putting a number

# Globally set Max rows displayed in output to 20
pd.set_option('display.max_rows', None)  # Great! No deprecation, many rows printed as wanted

# Globally set decimal places Displayed in output to any number of choice, change it prn
pd.set_option('display.precision', 2)

# Globally set floatto avoid getting such values in
#exponent notation, in correlation especially (Worked)
pd.options.display.float_format = '{:20,.4f}'.format 

# Globally set max_colwidth display
pd.options.display.max_colwidth = None # This removes any limit to num of col to be shown

# Globally set display expand_frame_repr
pd.set_option('display.expand_frame_repr', False)





#### Read the pickle file containing the comparison output to memory 

**** Comment out other datasets while reading one that you are to work with 

In [ ]:
##This is the comparison output data where the blocking variable was firstORlastname
pprl_DF2=pd.read_pickle('comparison_output_fln.pkl')# dubbed as "firstORlastname" dataset


##This is the comparison output data where the blocking variable was firstname
pprl_DF2=pd.read_pickle('comparison_output_fn.pkl') # dubbed as "firstname" dataset


##This is the comparison output data where the blocking variable was lastname
pprl_DF2=pd.read_pickle('comparison_output_ln.pkl')# dubbed as "lastname" dataset


### Exploratory Data Analysis 

In [ ]:
pprl_DF2.info()


In [ ]:
pprl_DF2.columns

#### Rename columns as needed, using Dictionary data structure

In [ ]:
pprl_DF2.rename(columns={'dob': 'dateOfBirth', 'firstname':'firstName','lasttname':'lastName'}, inplace=True)

In [ ]:
pprl_DF2.info()

### Multiclass Classification Target Engineering

In [ ]:
## Remove the binary target

pprl_DF2.drop("true_links",axis=1, inplace=True)

### Compute Df rowwise mean, and round to 1 decimal place to permit float comparison down the road

In [ ]:
#Compute Df rowwise mean, and round to 1 decimal place to permit float comparison down the road
pprl_DF2['row_Means'] =pprl_DF2.mean(axis=1).round(1) # Round to permit comparison, based on exp with working wt mock data
pprl_DF2['row_Means'].describe()
pprl_DF2['row_Means'].head()
print('============================')
pprl_DF2.info()


#### Bin the row means and categorize them as matching classes

In [ ]:
# #### View distinct values in matched DF and their frequencies, and sort them by frequency

pprl_DF2['row_Means'].value_counts().sort_values()

pprl_DF2['matched']=pprl_DF2['row_Means']


pprl_DF2['matched']=pd.cut(pprl_DF2.matched, bins=[
    -0.001, 0.45, 0.75,2.0], labels=['no','mayBe','yes'])# small, medium, large


In [ ]:
pprl_DF2.drop('row_Means', axis=1, inplace=True)

##### Create feature vectors 

In [ ]:
x =pprl_DF2.drop("matched", axis=1)
y =pprl_DF2.matched

###### Transform categorical data into numerical data
(only the target is categorical in the 3 cases)

In [ ]:
label_encod=LabelEncoder()
label_encod.fit(y)
y=label_encod.transform(y)
classes=label_encod.classes_
classes

In [ ]:
x.shape
y.shape

##### Import the modelling-related classes


In [ ]:
#!pip install imblearn # Required before importing the imblearn pipeline and SMOTE 


In [ ]:

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

from sklearn.multiclass import OneVsRestClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report # useful for evaluating our models, 

from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline as imbaPipe # The SMOTE class is in the imblearn package.
from imblearn.pipeline import make_pipeline # For gridsearching SMOTE inside Grid


from imblearn.over_sampling import SMOTE # SMOTE= synthetic minority oversampling technique
from imblearn.metrics import classification_report_imbalanced

from sklearn.metrics import recall_score, f1_score, precision_score,roc_curve
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support # For summary table


### Split the dataset (Create train-test data)

In [ ]:
## Split Data into training and test (you can split as desired, but 70:30% was used here) 

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3,random_state=42)


In [ ]:
##### Check data shape after spliting, before scaling

In [ ]:
# Confirm the Output Dimensions
# We can confirm the dimensions of the data are the same within test and train
# The proportion should also be close to the test_size argument.


x_train.shape
x_test.shape
y_train.shape
y_test.shape

#### Now scale the training and testing features separately to avoid data leakage

In [ ]:
scaler = MinMaxScaler() # If StandardScaler is used, there could be negative numbers generated and some ML algs may not work well with -ve numbers

# TO AVOID DATA LEAKAGE, SCALE EACH PART OF THE TRAINING DATA SEPARATELY
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

##### It is expected that scaling put all the trainign data in a range of 0 and 1 by now-- yes, achieved!

x_train[:5]# Show me 5 records in Xtrain
x_test[:5] # Show me 5 records in Xtest

In [ ]:
## Import the necessary modeling packages for multiclass classification

In [ ]:
from sklearn import pipeline# import the pipeline class
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from imblearn.pipeline import Pipeline as imbaPipe

from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier # OVR
from sklearn.multiclass import OneVsOneClassifier # OVO
from sklearn import svm # Required with OVR
from sklearn.linear_model import LogisticRegression# its multinomial parameter is ovr by default
from sklearn.naive_bayes import MultinomialNB# its multinomial parameter is ovr by default

from sklearn import svm


# Import the Pipeline API

from sklearn.pipeline import Pipeline
# Import a data standaizer of your choice

from sklearn.preprocessing import MinMaxScaler  # To handle outliers etc
from sklearn.preprocessing import LabelEncoder,     OneHotEncoder, MinMaxScaler, StandardScaler # aka ONEHOT: for transforming Cat features (X) to Numerical
import sklearn

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score


from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from sklearn.metrics import roc_auc_score
# import the needed module from scikit
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support # For summary table


#### Perform ML Experiments 

#### Build multiple models
***** Steps:

1. First: Build baseline models
2. Address class imbalance- using SMOTE (first in default mode with classifiers in their default modes as well)
3. Perform hyperparameter tuning of classifiers and SMOTE to improve model performance
4. Build more models
5. Evaluate the models
6. Identify the most generalizable model

### Feature Selection 
Carry out feature selection before data augmentation because the latter operation could influence the result of the former.

Feature selection is a process of eliminating the less/non-predictive features from the data. The benefits of this process include dimension reduction, model performance improvement, and 

#### Information Gain Analysis (Feature importances) with random forest (prior data augmentation)

## ***** Add note on information gain analysis



In [ ]:
#### 

In [ ]:

rForestPipe_SK=Pipeline([("dataScaler", scaler),('forest', RandomForestClassifier())])
Forestmodel=rForestPipe_SK.steps[1][1].fit(x_train, y_train)


In [ ]:
df_OfFeat_Importance=pd.DataFrame({'PPRL_ForLastname_Coef': x.columns, 
                                   "Rforest Relative Importance":Forestmodel.feature_importances_}
                                 ).sort_values(
                        "Rforest Relative Importance", ascending=False)



df_OfFeat_Importance.to_csv('PPRL_featImp_BVar_[FirstORlastname].csv')
df_OfFeat_Importance

#### Plot feat importance

In [ ]:
feat_Importance=Forestmodel.feature_importances_
sorted_index=np.argsort(feat_Importance)
#position=np.arange(sorted_index.shape)# output is a tuple:  array([0, 3, 2, 1], dtype=int64)
position1=np.arange(sorted_index.shape[0])# we only need the item in index 0 of the tuple
 
position2=np.arange(sorted_index.shape[0])+0.5 # add 0.5 else, it'll start at 0
sorted_index=np.argsort(feat_Importance)
position=np.arange(sorted_index.shape[0])+0.5
fig=plt.figure(figsize=(12,6))
#plt.subplot(1,2,1)
plt.barh(position,feat_Importance[sorted_index])#, align="center")
plt.yticks(position,np.array(x.columns)[sorted_index])
#plt.bar_label(plt.containers[0])
plt.title("PPRL Feature Importance: BVar [FirstORlastname]")
#plt.savefig("PPRL_featImp_FirstORlastname.png")


**** Set information gain threshold (IGT), and drop any feature not meeting the threshold
Example: For the "firstORlastname" dataset, IGT was set to 0.1, without approximation), and the feature not meeting this threshold was dropped (as shown below)

In [ ]:
pprl_DF2.drop('zipCode', axis=1, inplace=True)# based on feature importances result

### Parameter Tuning for "FirstORlastname" dataset


#### NBayes 

In [ ]:
# # Make an instance of the NBayes (Put all params at default)

nBayes_Instance=MultinomialNB()
## Obtain the parameters available inside NBayes

#nBayes_Instance.get_params().keys()

# Output: dict_keys(['alpha', 'class_prior', fit_prior'])


crossVal = KFold(n_splits=3, random_state=2, shuffle=True)

nBayesparam_gridTosearch={'alpha': [1,2,3,5,5,7,8,9,10]}

grid_NBayes = GridSearchCV(nBayes_Instance,
                         param_grid = [nBayesparam_gridTosearch],
                         cv=crossVal,
                         scoring='roc_auc',
                         return_train_score=True,
                         n_jobs=-1,
                         verbose=True)

grid_NBayes.fit(x_train, y_train)
NBbestimator = grid_NBayes.best_estimator_
NBbestimator
ypred = NBbestimator.predict(x_test)# predicted_Classes
print(ypred)


*****Utilize the obtained result in searching for the best k value in the k_neighbor parameter of SMOTE for NBayes

In [ ]:

for k in range(1,11): # 10 k values

    smDefauNbayes_def= imbaPipe([('smote', SMOTE(sampling_strategy ="auto",
                                            k_neighbors=k, random_state=40)),
                                           ("dataScaler", scaler), 
                                 ("naive", MultinomialNB(alpha=1))])

    pipe1=smDefauNbayes_def
         #### RESAMPLE THE DATA NOW, SINCE SPLITED INTO FOLDS 
#     x_train_oversampled, y_train_oversampled = pipe1.steps[0][1].fit_resample(
#                                                                      x_train, y_train)
    
    x_train_oversampled=x_train
    y_train_oversampled =y_train  
    
    nbayes_model=pipe1.fit(x_train_oversampled,y_train_oversampled)## see lecture 6.7 from Andrew ng on log reg for multi class ovr
    
    #    nbayes_model=pipe1.steps[2][1].fit(x_train_oversampled,y_train_oversampled)## see lecture 6.7 from Andrew ng on log reg for multi class ovr

    predicted_Values=nbayes_model.predict(x_test)
    train_testScore=(nbayes_model.score(x_train_oversampled,
                                        y_train_oversampled),
                                     nbayes_model.score(x_test,y_test))
    print('> k=%d', (k, train_testScore))

##### Use the output to create full Imblearn Pipeline for Naive Bayes in each modeling file

#### Random Forest (an estimator for OVR)

In [ ]:

# Make an instance of the logisitic regression (Put all params at default, hence, left empty)
randForest_Instance = RandomForestClassifier()## or pass .steps[1][1]
## Create a Dict of param names and list their values as available in Log Reg Scikit
   
#RandForest_Instance.get_params().keys()

## Number of trees in the forest. We're using list comprehension here
n_estimators=[int(x) for x in np.linspace(start=2, stop=5, num=12)] #  i.e 12 of these, starting from 10, up to 80

## number of features to considere at every node split 
max_features= ['auto', 'sqrt']

## Maximum of levels in each tree
max_depth= [4, 8, 12, 25]

random_state= [1,4, 20, 42]


# Minimum number lof samples required to split a node
min_samples_split= [2, 5,10]

## Minimum number of samples required at each leaf node
min_samples_leaf= [5, 10, 20, 25]

bootstrap= [True, False]

resampler = SMOTE(sampling_strategy="auto", random_state=42,k_neighbors=5)

           
### NOW CREATE A DICT OF THE PARAMS IN THE PARAMS GRID            
RFparam_gridTosearch= {'n_estimators': n_estimators, 
            'max_features': max_features,
             'max_depth' : max_depth,
              'random_state' : random_state,
            'min_samples_split' : min_samples_split,
             'min_samples_leaf' : min_samples_leaf,
            'bootstrap': bootstrap}

################
print('Let us see what we have inside the gridsearch algorithm', RFparam_gridTosearch)          

#cv=2,


gridSch_ForRandForest =GridSearchCV(estimator=randForest_Instance,
                                    param_grid = RFparam_gridTosearch, cv=3,
                     verbose=2,n_jobs=-1)


##### Now fit the training set on Grid

BestClf_fromGrid_resultRF = gridSch_ForRandForest.fit(x_train,y_train)

print("============== get best params- is it not the same as get best estimator_"  )
BestClf_fromGrid_resultRF.best_params_

#output of
# get BestClf_fromGrid_resultRF.best_params_ 
                                                # {'bootstrap': False, 'max_depth': 25, 
#                                             'max_features': 'auto', 'min_samples_leaf': 5, 
#                                             'min_samples_split': 2, 'n_estimators': 35}


print('get BestClf_fromGrid_resultRF.best_params_', BestClf_fromGrid_resultRF.best_params_)

BestClf_fromGrid_resultRF.best_estimator_


# Check accuracy of the GridSeatch model
print('BestClf_fromGrid_resultRF.score', BestClf_fromGrid_resultRF.score(x_test,y_test))

print("new formating way, see if the printing works")
print(f'Train Accu_RForest_WtGridSch - : {BestClf_fromGrid_resultRF.score(x_train,y_train):.3f}')
print(f'TestAccu_RForest_WtGridSch - : {BestClf_fromGrid_resultRF.score(x_test,y_test):.3f}')



*****Utilize the obtained result in searching for the best k value in the k_neighbor parameter of SMOTE for OVR

In [ ]:
for k in range(1,11): # 10 k values
    smOVRrfPipe_def= imbaPipe([('smote', SMOTE(sampling_strategy ="auto",
                                        k_neighbors=k, random_state=40)),
                                    ("dataScaler", scaler),
                               ('ovrForest', OneVsRestClassifier(RandomForestClassifier(
                                   bootstrap =False,max_depth=25,min_samples_leaf= 5,
                                    min_samples_split=2,n_estimators=5,random_state= 42)))])  
    pipe2=smOVRrfPipe_def  
    x_train_oversampled=x_train
    y_train_oversampled =y_train  
    
    ovr_model=pipe2.fit(x_train_oversampled,y_train_oversampled)## see lecture 6.7 from Andrew ng on log reg for multi class ovr

    
    predicted_Values=ovr_model.predict(x_test)
    train_testScore=(ovr_model.score(x_train_oversampled,
                    y_train_oversampled),
                     ovr_model.score(x_test,y_test))
    print('> k=%d', (k, train_testScore))

##### Use the output to create full Imblearn Pipeline for OneVersusRest.


#### Logistic Regression Optimization

In [ ]:
classifier=LogisticRegression()
crossVal = KFold(n_splits=3, random_state=2, shuffle=True)

LRegparam_gridTosearch = {'penalty': ['l2'], 'multi_class': ['auto','ovr'],
              'solver': ['newton-cg','lbfgs', 'sag', 'saga'], 
                          'C': [0.001,0.01,0.1,1,10,100,1000]}

grid_Lreg = GridSearchCV(classifier,param_grid = LRegparam_gridTosearch,
                         cv=crossVal,
                         scoring='accuracy',
                         return_train_score=True,
                         n_jobs=-1,
                         verbose=True
                        )
grid_Lreg.fit(x_train, y_train)
bestimator = grid_Lreg.best_estimator_
bestimator
ypred = bestimator.predict(x_test)# predicted_Classes
print(ypred)


In [ ]:
for k in range(1,11): # 10 k values
    smLRegPipe_def= imbaPipe([('smote', SMOTE(sampling_strategy ="auto", 
                                              k_neighbors=k, random_state=40)),
                                           ("dataScaler", scaler),
                              ("lreg", LogisticRegression(C=1000, solver='newton-cg'))])
    pipe4=smLRegPipe_def
    x_train_oversampled=x_train
    y_train_oversampled =y_train  
    
    LReg_model=pipe4.fit(x_train_oversampled,y_train_oversampled)## see lecture 6.7 from Andrew ng on log reg for multi class ovr

    
    predicted_Values=LReg_model.predict(x_test)
    train_testScore=(LReg_model.score(x_train_oversampled,y_train_oversampled),
                     LReg_model.score(x_test,y_test))
    print('> k=%d', (k, train_testScore))

##### Use the output to create full Imblearn Pipeline for Naive Bayes in each modeling file for LReg


In [ ]:
print("Done with feature selection and parameter tuning for firstORlastname [fln] data!=====")